In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [0]:
#these file already preprocessing
data = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/DeepWork_216/master/train_data.csv')
test_X = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/DeepWork_216/master/test_data.csv')

In [0]:
#prepare variable
y = data.income
X = data.drop(['income', 'id'], axis=1)


X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

#seperate column by type
categorical_cols = [cname for cname in ['gender','ocp_cd', 'age']]
numerical_cols = [cname for cname in X if cname not in categorical_cols]

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [9]:
data.head()

,id,gender,ocp_cd,age,cc_sum,cc_mean,cc_cnt_sum,cc_cnt_mean,kp_mean,kp_sum,kp_cnt_mean,kp_cnt_sum,overall,income
0,1,2,9.0,5,44700.0,7450.000000,5.0,0.833333,291.666667,1.750000e+03,0.166667,1.000000,4.645000e+04,20000
1,2,2,3.0,4,65200.0,10866.666667,42.0,7.000000,8221.666667,4.933000e+04,4.108333,24.650000,1.145300e+05,106000
2,3,2,1.0,3,0.0,0.000000,0.0,0.000000,6713.888889,4.028333e+04,3.388889,20.333333,4.028333e+04,29000
3,4,2,3.0,5,770000.0,128333.333333,36.0,6.000000,242291.666667,1.453750e+06,12.675000,76.050000,2.223750e+06,61000
4,5,2,9.0,4,12000.0,2000.000000,3.0,0.500000,0.000000,0.000000e+00,0.000000,0.000000,1.200000e+04,18000


In [10]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

# Create XGB Classifier object
xgb_reg = xgb.XGBRegressor(tree_method = "exact", predictor = "cpu_predictor", verbosity = 1,
                           eval_metric = ["mae"], objective = "reg:squarederror")
# Create parameter grid
parameters = {"learning_rate": [0.1, 0.01, 0.001],
               "gamma" : [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2],
               "max_depth": [2, 4, 7, 10],
               "colsample_bytree": [0.3, 0.6, 0.8, 1.0],
               "subsample": [0.2, 0.4, 0.5, 0.6, 0.7],
               "reg_alpha": [0, 0.5, 1],
               "reg_lambda": [1, 1.5, 2, 3, 4.5],
               "min_child_weight": [1, 3, 5, 7],
               "n_estimators": [100, 250, 500, 1000]}

# Create RandomizedSearchCV Object
xgb_rscv = RandomizedSearchCV(xgb_reg, param_distributions = parameters, scoring = "neg_mean_absolute_error",
                             cv = 7, verbose = 3, random_state = 40)
#pipeline
rscv_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', xgb_rscv)
                             ])
# Fit the model
model_xgboost = rscv_pipeline.fit(X_train, y_train)

# Model best estimators
print("Learning Rate: ", model_xgboost.named_steps['model'].best_estimator_.get_params()["learning_rate"])
print("Gamma: ", model_xgboost.named_steps['model'].best_estimator_.get_params()["gamma"])
print("Max Depth: ", model_xgboost.named_steps['model'].best_estimator_.get_params()["max_depth"])
print("Subsample: ", model_xgboost.named_steps['model'].best_estimator_.get_params()["subsample"])
print("Max Features at Split: ", model_xgboost.named_steps['model'].best_estimator_.get_params()["colsample_bytree"])
print("Alpha: ", model_xgboost.named_steps['model'].best_estimator_.get_params()["reg_alpha"])
print("Lamda: ", model_xgboost.named_steps['model'].best_estimator_.get_params()["reg_lambda"])
print("Minimum Sum of the Instance Weight Hessian to Make a Child: ",
      model_xgboost.named_steps['model'].best_estimator_.get_params()["min_child_weight"])
print("Number of Trees: ", model_xgboost.named_steps['model'].best_estimator_.get_params()["n_estimators"])


Fitting 7 folds for each of 10 candidates, totalling 70 fits
[CV] subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0, score=-25448.035, total=   3.1s
[CV] subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0, score=-24916.605, total=   2.9s
[CV] subsample=0.6, reg_lambda=1, reg_alpha=0, n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.1, gamma=1.5, colsample_bytree=1.0 


...

[CV]  subsample=0.4, reg_lambda=2, reg_alpha=0.5, n_estimators=250, min_child_weight=7, max_depth=7, learning_rate=0.001, gamma=2, colsample_bytree=0.6, score=-36806.152, total=  14.6s
[CV] subsample=0.4, reg_lambda=2, reg_alpha=0.5, n_estimators=250, min_child_weight=7, max_depth=7, learning_rate=0.001, gamma=2, colsample_bytree=0.6 


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  subsample=0.4, reg_lambda=2, reg_alpha=0.5, n_estimators=250, min_child_weight=7, max_depth=7, learning_rate=0.001, gamma=2, colsample_bytree=0.6, score=-37979.914, total=  14.6s


[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed: 23.8min finished
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Learning Rate:  0.01
Gamma:  0.3
Max Depth:  10
Subsample:  0.7
Max Features at Split:  1.0
Alpha:  0
Lamda:  3
Minimum Sum of the Instance Weight Hessian to Make a Child:  7
Number of Trees:  250


Best parameter
Learning Rate:  0.01
Gamma:  0.3
Max Depth:  10
Subsample:  0.7
Max Features at Split:  1.0
Alpha:  0
Lamda:  3
Minimum Sum of the Instance Weight Hessian to Make a Child:  7
Number of Trees:  250

In [0]:
final_model = xgb.XGBRegressor(objective="reg:linear", random_state=42, 
                             learning_rate=0.01, gamma=0.3, max_depth=10, 
                             colsample_bytree=1, subsample=0.7, reg_alpha=0, reg_lambda=3,
                             min_child_weight=7, n_estimators=250
                             )

In [0]:
final_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', final_model)
                             ])

In [14]:
final_pipeline.fit(X_train, y_train)
preds = final_pipeline.predict(X_valid)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[11:14:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [15]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(final_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')
print("Average MAE score (across experiments):")
print(scores.mean())

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[11:15:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[11:16:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[11:16:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[11:17:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[11:18:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Average MAE score (across experiments):
23594.98399074219


In [16]:
'''
Model conclusion
Best parameter
Learning Rate:  0.01
Gamma:  0.3
Max Depth:  10
Subsample:  0.7
Max Features at Split:  1.0
Alpha:  0
Lamda:  3
Minimum Sum of the Instance Weight Hessian to Make a Child:  7
Number of Trees:  250
'''

'\nModel conclusion\nBest parameter\nLearning Rate:  0.01\nGamma:  0.3\nMax Depth:  10\nSubsample:  0.7\nMax Features at Split:  1.0\nAlpha:  0\nLamda:  3\nMinimum Sum of the Instance Weight Hessian to Make a Child:  7\nNumber of Trees:  250\n'